# Contrastive Learning From Scratch

An attempt to build contrastive learning model from scratch. Parts include:

- Loading and preparing Wiki-1M data for model input
- Contrastive learning model
  - Forward passing using pre-trained model
  - Constrastive layer
  - Calculate loss
- Training procedure
  - Default trainer optimizer
  - Default trainer hyper-parameters

In [23]:
import os

# Set Project home
PROJECT_HOME = os.path.join('/',
                            'workspace',
                            'gatech',
                            'cs7643-deep-learning',
                            'contrastive-learning-in-distilled-models')
%cd {PROJECT_HOME}

# Load project code
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, './src')

import distilface

/workspace/gatech/cs7643-deep-learning/contrastive-learning-in-distilled-models


## 1. Loading and Preparing Wiki-1M data

Use huggingface `datasets` library to load local file data.

In [25]:
import numpy as np

from datasets import load_dataset

data_files = {'train': 'data/training/wiki1m_for_simcse.txt'}
# data_files = {'train': 'data/training/wiki5k.txt'}
datasets = load_dataset('text', data_files=data_files, cache_dir='./data/')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to ./data/text/default-c294b10b923459c8/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
# Unsupervised / Self-supervised dataset

column_names = datasets["train"].column_names
sent0_cname = column_names[0]
sent1_cname = column_names[0]

print('column_names:', column_names)
print('sent0_cname:', sent0_cname, '| sent1_cname:', sent1_cname)

column_names: ['text']
sent0_cname: text | sent1_cname: text


In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [28]:
def prepare_features(examples):
    total = len(examples[sent0_cname])

    # Avoid "None" fields 
    for idx in range(total):
        if examples[sent0_cname][idx] is None:
            examples[sent0_cname][idx] = " "
        if examples[sent1_cname][idx] is None:
            examples[sent1_cname][idx] = " "
    
    sentences = examples[sent0_cname] + examples[sent1_cname]

    sent_features = tokenizer(
        sentences,
        max_length=32,
        truncation=True,
        padding=True,
    )

    features = {}
    for key in sent_features:
        features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]

    return features

In [29]:
train_dataset = datasets["train"].map(prepare_features,
                                      batched=True,
                                    #   num_proc=24,
                                      remove_columns=column_names)

  0%|          | 0/1000 [00:00<?, ?ba/s]

In [30]:
train_dataset.num_rows

1000000

In [31]:
str(train_dataset['input_ids'][0][0])

'[101, 26866, 1999, 2148, 2660, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [32]:
str(train_dataset['input_ids'][0][1])

'[101, 26866, 1999, 2148, 2660, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [33]:
train_dataset.features.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

Sentence 1 and Sentence 2 are the same sentence

## 2. Contrastive Learning Model

In [34]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, BertModel, BertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from distilface.modules.pooler import Pooler
from distilface.modules.similarity import Similarity


class BertCLModel(BertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_first_last', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.bert = BertModel(config, add_pooling_layer=False)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        if self.training:
            return self.cl_forward(self.bert, input_ids, attention_mask, token_type_ids)
        else:
            return self.sent_emb(self.bert, input_ids, attention_mask, token_type_ids)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None, token_type_ids=None):
        batch_size = input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)
        token_type_ids = token_type_ids.view((-1, token_type_ids.size(-1))) # (bs * num_sent, len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_attentions=True,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None, token_type_ids=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_attentions=True,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'bert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

model = BertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

model.eval();


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertCLModel: ['cls.predictions.transform.dense.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 2.1 Initial BERT embeddings performance

In [35]:
import senteval


def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**batch)
    model.train()

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    results = se.eval(tasks)

    print('STS12: ', results["STS12"]["all"]["spearman"]["all"])
    print('STS13: ', results["STS13"]["all"]["spearman"]["all"])
    print('STS14: ', results["STS14"]["all"]["spearman"]["all"])
    print('STS15: ', results["STS15"]["all"]["spearman"]["all"])
    print('STSB: ', results["STSBenchmark"]["test"]["spearman"][0])

    return results

In [36]:
results = evaluate_model()
results

/opt/conda/envs/cl-distilled/lib/python3.8/site-packages/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/opt/conda/envs/cl-distilled/lib/python3.8/site-packages/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


KeyError: 'all'

## 3. Trainer

In [14]:
import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=5e-05,
    weight_decay=0.0,
    num_train_epochs=1,
    eval_steps=100,
)

model.train()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

In [15]:
model_path = 'trained_model/bert_cl'

train_result = trainer.train(model_path=model_path)

Step,Training Loss


KeyboardInterrupt: 

## 4. Evaluate Bert CL Model performance

In [17]:
model.eval()

results = evaluate_model()
results

STS12:  0.5675054534807098
STS13:  0.7281554777556237
STS14:  0.6335945639499094
STS15:  0.7600618289222154
STSB:  0.7113600318308351


{'STSBenchmark': {'train': {'pearson': (0.7414994287646051, 0.0),
   'spearman': SpearmanrResult(correlation=0.7102657603877065, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7561389569108177, 3.798134217661319e-278),
   'spearman': SpearmanrResult(correlation=0.7619754084096394, pvalue=5.964770119842861e-285),
   'nsamples': 1500},
  'test': {'pearson': (0.7272632396921386, 2.519373640229751e-227),
   'spearman': SpearmanrResult(correlation=0.7113600318308351, pvalue=3.899147457192333e-213),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.7409343456431441,
    'mean': 0.7416338751225204,
    'wmean': 0.7417692001471257},
   'spearman': {'all': 0.7242768878083036,
    'mean': 0.7278670668760604,
    'wmean': 0.7194305114717322}}},
 'STS12': {'MSRpar': {'pearson': (0.4222923718591043, 8.57578903746429e-34),
   'spearman': SpearmanrResult(correlation=0.44805503021367254, pvalue=2.5934246235314064e-38),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.8216320722327597, 9.